In [10]:
# get the status of the bridges
import tweepy 
import time
import csv
from datetime import datetime
from config import *

In [34]:
import pandas as pd

In [157]:
df = pd.read_csv("../data/bridge/bridge_status.csv")
df = df.sort(['date','time'])

df['timeobj'] = pd.to_datetime(df['timeobj'])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [158]:
# Get a list of bridges that opened but have not closed

# All closed bridges should have an open even followed by a later closed event

# Check all the close events
close_events = df[df['event'] == 'closed'].drop_duplicates(subset='bridge', keep='first')
open_events = df[df['event'] == 'open'].drop_duplicates(subset='bridge', keep='first')

In [159]:
open_events['timeobj']

3224   1900-01-01 11:01:09
3219   1900-01-01 11:56:28
3222   1900-01-01 11:21:41
3211   1900-01-01 12:41:53
3204   1900-01-01 13:16:49
3140   1900-01-01 10:53:52
3098   1900-01-01 11:12:46
Name: timeobj, dtype: datetime64[ns]

In [160]:
# get only the most recent open and close events
df = pd.merge(close_events,open_events,on='bridge',suffixes=['_close','_open'])

In [187]:
# fake some data for testing - set the last close time to a later date
df['timeobj_close'].iloc[0] = df['timeobj_close'].iloc[0].replace(hour=12)

In [189]:
current_closures = df[(df['date_open'] != df['date_close']) | (df['timeobj_open'] < df['timeobj_close'])]

In [190]:
current_closures

,bridge,time_close,event_close,date_close,text_close,newtime_close,timeobj_close,time_open,event_open,date_open,text_open,newtime_open,timeobj_open
0,Ballard,10:45:26 AM,closed,2015-11-09,The Ballard Bridge has closed to traffic - 1...,10:45:26 AM,1900-01-01 12:45:26,11:01:09 AM,open,2015-11-09,The Ballard Bridge has reopened to traffic - ...,11:01:09 AM,1900-01-01 11:01:09


In [192]:
# Map each closure
# provide an expected window of re-opening based on historical closure data
# show time closed, bell curve of likely re-opening

In [193]:
# Create some leaflet thing here
# try folium: https://github.com/python-visualization/folium

In [253]:
df['test'] = df.index

In [272]:
import folium
import vincent
vincent.core.initialize_notebook()

In [302]:
map_osm = folium.Map(location=[47.5836, -122.3750], zoom_start=12, tiles='Stamen Toner')
# folium.Marker([47.570950, -122.348324], popup='Spokane Street').add_to(map_osm)
# folium.Marker([47.529252, -122.314316], popup='South Park').add_to(map_osm)

# get fancy 
# see e.g., http://nbviewer.jupyter.org/github/ocefpaf/folium_notebooks/blob/master/test_png_mpld3_vega_popup.ipynb
# show some fake data
resolution, width, height = 50, 7, 3
vis = vincent.Line(df['test'], width=width*resolution, height=height*resolution)
vis.legend(title='Spokane');

vega = folium.Vega(json.loads(vis.to_json()), width="30%", height="30%")
popup = folium.Popup(max_width=vis.width+75).add_child(vega)
icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[47.570950, -122.348324], popup=popup, icon=icon)

map_osm.add_children(marker);

map_osm.save('osm.html')

In [304]:
map_osm

In [ ]:
# pop window should show time elapsed since road closure
# maybe as a progress bar
# or estimated time remaining open based on past records & ship data
# 